In [13]:
import networkx as nx
import numpy as np
import pandas as pd

from cascade_failure.updated_model import load_data, assign_atribute
from src.data import load_data_with_attributes

In [14]:
G=load_data_with_attributes()

node=8

assert node in G.nodes(), f" Node {node} not found in graph"

Graph loaded: 4941 nodes, 6594 edges


In [15]:
# centrality measures

degree=dict(G.degree())
betweenness=nx.betweenness_centrality(G, normalized=True)
closeness=nx.closeness_centrality(G)

node_metrics={
    "degree": degree[node],
    "betweenness": betweenness[node],
    "closeness": closeness[node]
}

print("Centrality metrics for node 8:")
for k, v in node_metrics.items():
    print(f"{k:12s}: {v:.6f}")

Centrality metrics for node 8:
degree      : 3.000000
betweenness : 0.000810
closeness   : 0.058241


In [16]:
# percentile ranking
def percentile_rank(values, x):
    return 100.0*sum(v<=x for v in values) / len(values)

print("\nPercentile ranks:")
print(f"Degree: {percentile_rank(degree.values(), degree[node]):.2f}%")
print(f"Betweenness:  {percentile_rank(betweenness.values(), betweenness[node]):.2f}%")
print(f"Closeness:    {percentile_rank(closeness.values(), closeness[node]):.2f}%")


Percentile ranks:
Degree: 79.78%
Betweenness:  64.34%
Closeness:    73.30%


In [17]:
# articulation point test
articulation_points=set(nx.articulation_points(G))
is_articulation=node in articulation_points

print("\nArticulation point:")
print(f"Node 8 is articulation point: {is_articulation}")


Articulation point:
Node 8 is articulation point: True


In [18]:
# network after removing node 8:

def largest_component_fraction(graph):
    if graph.number_of_nodes == 0:
        return 0
    largest_cc = max(nx.connected_components(graph), key=len)
    return len(largest_cc) / graph.number_of_nodes()

original_lcc = largest_component_fraction(G)

G_removed= G.copy()
G_removed.remove_node(node)
node8_lcc= largest_component_fraction(G_removed)

print("\nLargest connected component:")
print(f"Original network: {original_lcc:.4f}")
print(f"After removing Node 8: {node8_lcc:.4f}")
print(f"Drop caused by Node 8: {(original_lcc - node8_lcc):.4f}")

# compare to random node removal

rng= np.random.default_rng(42)
random_nodes=rng.choice(list(G.nodes()), size=50, replace=False)

random_lcc_drops = []
for n in random_nodes:
    if n==node:
        continue
    H = G.copy()
    H.remove_node(n)
    random_lcc_drops.append(original_lcc - largest_component_fraction(H))

print("\nRandom node comparison:")
print(f"Average random drop: {np.mean(random_lcc_drops):.6f}")
print(f"Max random drop:     {np.max(random_lcc_drops):.6f}")
print(f"Node 8 drop:         {(original_lcc - node8_lcc):.6f}")


Largest connected component:
Original network: 1.0000
After removing Node 8: 0.9996
Drop caused by Node 8: 0.0004

Random node comparison:
Average random drop: 0.000206
Max random drop:     0.002024
Node 8 drop:         0.000405


In [ ]:
print("\nLoad-capacity diagnostics:")
print(f"Load:     {G.nodes[node]['load']}")
print(f"Capacity: {G.nodes[node]['capacity']}")
print(f"Utilization ratio: {G.nodes[node]['load'] / G.nodes[node]['capacity']:.4f}")


Load–capacity diagnostics:
Load:     20.885424887303657
Capacity: 36.60728621977991
Utilization ratio: 0.5705
